In [1]:
import pandas as pd
from pytube import YouTube, Playlist
import pydub
import os
import re
import glob

os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

import moviepy.editor as mpeditor
import moviepy.video.fx.all as mpfx

target_dir = "./all_data"

In [2]:
if not os.path.isdir(target_dir + "/mp4"):
    os.makedirs(target_dir + "/mp4")
if not os.path.isdir(target_dir + "/wav"):
    os.makedirs(target_dir + "/wav")



traincsv = open(target_dir + "/train.csv", "w")
traincsv.write("\"file\",\"Description\"\n")
failfile = open(target_dir + "/fail.txt", "w")
list_url = "https://www.youtube.com/playlist?list=PL96kIIcXJpMtmsQGlsNVqWduASZnh4HnE"
playlist = Playlist(list_url)

idx = 0
for video in playlist.videos:

    if idx == 1:
        break

    tmp_vidoe = target_dir + "/mp4/tmp" + str(idx) + ".mp4"
    mp4_path = target_dir + "/mp4/" + str(idx) + ".mp4"
    wav_path = target_dir + "/wav/" + str(idx) + ".wav"

    if not os.path.isfile(mp4_path):
        try:
            video.streams.filter().get_highest_resolution().download(filename="tmp" + str(idx) + ".mp4", output_path=target_dir + "/mp4")
            mp4 = mpeditor.VideoFileClip(tmp_vidoe)
            mp4 = mpfx.crop(mp4, x1=200, y1=565, width=900, height=50)
            mp4.write_videofile(filename=mp4_path, fps=10, logger=None)
            os.remove(tmp_vidoe)

        except:
            print(idx, video.title, "cant downlaod")
            failfile.write(str(idx) + ". " + "cant downlaod.\n")
            continue
    
    

    if not os.path.isfile(wav_path):
        try:
            audio = mpeditor.AudioFileClip(mp4_path, logger=None)
            audio.write_audiofile(filename=wav_path, fps=16000, nbytes=2, logger=None)
            sound = pydub.AudioSegment.from_wav(wav_path)
            sound = sound.set_channels(1)
            sound.export(wav_path, format="wav")
        except:
            print(idx, "cant convert")
            failfile.write(str(idx) + ". " + "cant convert.\n")
            continue

    print(idx, "success")
    idx += 1

traincsv.close()
failfile.close()
    



0 success
